In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from flask import Flask, render_template
import psycopg2
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_file, show
import matplotlib.pyplot as plt


In [2]:
t_host = "provisionaldb2.cpvxmi357s0k.us-east-2.rds.amazonaws.com" # either "localhost", a domain name, or an IP address.
t_port = "5432" # default postgres port
t_dbname = "GroupProjectDB"
t_user = "postgres"
t_pw = "postgres"
db_conn = psycopg2.connect(host=t_host, port=t_port, dbname=t_dbname, user=t_user, password=t_pw)
db_cursor = db_conn.cursor()

In [3]:
# Read data from PostgreSQL database table and load into a DataFrame instance
DashboardDataDF =  pd.read_sql("select * from \"turnoutanalysisdata\"", db_conn)
PercentRegisteredData =  [DashboardDataDF["electionyear"],DashboardDataDF["stateabbreviation"], DashboardDataDF["statename"] , DashboardDataDF["pct_reg_of_vep_vrs"],DashboardDataDF["voterturnout"]]
PercentRegisteredHeaders = ["ElectionYear","StateAbbreviation","StateName","PercentOfRegisteredVoters","PercentTurnout" ]
PercentRegisteredDF = pd.concat(PercentRegisteredData, axis=1, keys=PercentRegisteredHeaders)

In [4]:
PercentRegisteredDF

,ElectionYear,StateAbbreviation,StateName,PercentOfRegisteredVoters,PercentTurnout
0,2008,AL,Alabama,0.852208,0.608
1,2008,AK,Alaska,0.875693,0.680
2,2008,AZ,Arizona,0.829416,0.567
3,2008,AR,Arkansas,0.742385,0.525
4,2008,CA,California,0.824846,0.609
...,...,...,...,...,...
296,2018,VA,Virginia,0.874413,0.544
297,2018,WA,Washington,0.855181,0.583
298,2018,WV,West Virginia,0.778394,0.425
299,2018,WI,Wisconsin,0.828831,0.614


In [5]:
PercentRegisteredDF2008=PercentRegisteredDF[PercentRegisteredDF['ElectionYear']==2008]

In [6]:
PercentRegisteredDF2008

,ElectionYear,StateAbbreviation,StateName,PercentOfRegisteredVoters,PercentTurnout
0,2008,AL,Alabama,0.852208,0.608
1,2008,AK,Alaska,0.875693,0.680
2,2008,AZ,Arizona,0.829416,0.567
3,2008,AR,Arkansas,0.742385,0.525
4,2008,CA,California,0.824846,0.609
5,2008,CO,Colorado,0.840033,0.710
6,2008,CT,Connecticut,0.855638,0.666
7,2008,DE,Delaware,0.857940,0.656
8,2008,FL,Florida,0.876890,0.661
9,2008,GA,Georgia,0.849353,0.625


In [11]:
import plotly.express as pe

In [8]:
PercentRegisteredData =  [DashboardDataDF["electionyear"],DashboardDataDF["stateabbreviation"], DashboardDataDF["statename"] , DashboardDataDF["pct_reg_of_vep_vrs"], DashboardDataDF["voterturnout"]]

In [9]:
PercentRegisteredData

[0      2008
 1      2008
 2      2008
 3      2008
 4      2008
        ... 
 296    2018
 297    2018
 298    2018
 299    2018
 300    2018
 Name: electionyear, Length: 301, dtype: int64,
 0      AL
 1      AK
 2      AZ
 3      AR
 4      CA
        ..
 296    VA
 297    WA
 298    WV
 299    WI
 300    WY
 Name: stateabbreviation, Length: 301, dtype: object,
 0            Alabama
 1             Alaska
 2            Arizona
 3           Arkansas
 4         California
            ...      
 296         Virginia
 297       Washington
 298    West Virginia
 299        Wisconsin
 300          Wyoming
 Name: statename, Length: 301, dtype: object,
 0      0.852208
 1      0.875693
 2      0.829416
 3      0.742385
 4      0.824846
          ...   
 296    0.874413
 297    0.855181
 298    0.778394
 299    0.828831
 300    0.750195
 Name: pct_reg_of_vep_vrs, Length: 301, dtype: float64,
 0      0.608
 1      0.680
 2      0.567
 3      0.525
 4      0.609
        ...  
 296    0.544
 297 

In [12]:
PercentTurnoutDF2008 = PercentRegisteredDF2008

In [19]:
Turnout2008 = pe.bar(PercentTurnoutDF2008, x="StateAbbreviation", y="PercentTurnout",
             barmode='group',
             height=400, width=1000)
Turnout2008.show()

In [20]:
import plotly.graph_objects as go

fig = go.Figure(data=PercentTurnoutDF2008[
    go.Bar(name='Percent Turnout', x="StateAbbreviation", y="PercentTurnout",
    go.Bar(name='Percent Registered', x="StateAbbreviation", y="PercentOfRegisteredVoters")
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

SyntaxError: invalid syntax (<ipython-input-20-ace6bd1ef2ea>, line 6)